In [ ]:
!pip install keras


: 

In [54]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D,Conv2DTranspose,Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


In [55]:
def conv_block(x, n_base, batch_normalization):
    
    x = Conv2D(filters=n_base, kernel_size=(3,3), 
                        strides=(1,1),padding='same')(x)
    if (batch_normalization):
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=n_base, kernel_size=(3,3), 
                        strides=(1,1),padding='same')(x)
    if (batch_normalization):
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

In [56]:
def downsample_block(x, n_base, batch_normalization, dropout):
    f = conv_block(x, n_base, batch_normalization)
    p = layers.MaxPool2D(pool_size = (2,2))(f)
    if(dropout):
        p = layers.Dropout(0.2)(p)
        
    return f, p

In [57]:
def upsample_block(x, f, n_base, batch_normalization, dropout):
    
    x = Conv2DTranspose(filters=n_base, kernel_size=(2,2), 
                         strides=(2,2),padding='same')(x)
    x = Concatenate()([x,f])
    if(dropout):
        x = layers.Dropout(0.2)(x)
    x = conv_block(x, n_base, batch_normalization)
        
    return x

In [58]:
def get_unet(img_w, img_h, img_ch, n_base, LR, batch_normalization, dropout):

    
    
    ## Encoder part
#     model = Sequential()
    inputs = layers.Input((img_w, img_h, img_ch))
    
    f1, p1 = downsample_block(inputs, n_base, batch_normalization, dropout)
    f2, p2 = downsample_block(p1, n_base*2, batch_normalization, dropout)
    f3, p3 = downsample_block(p2, n_base*4, batch_normalization, dropout)
    f4, p4 = downsample_block(p3, n_base*8, batch_normalization, dropout)
    
    
    ## Bottleneck
    bottleneck = conv_block(p4, n_base*16, batch_normalization)
    
    ## Decoder part
    p5 = upsample_block(bottleneck, f4, n_base*8, batch_normalization, dropout)
    p6 = upsample_block(p5, f3, n_base*4, batch_normalization, dropout)
    p7 = upsample_block(p6, f2, n_base*2, batch_normalization, dropout)
    p8 = upsample_block(p7, f1, n_base, batch_normalization, dropout)

    
    ## 1 Convo layer
    p9 = Conv2D(filters=1, kernel_size=(1,1), 
                            padding='same')(p8)
    outputs = Activation('sigmoid')(p9)
    

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.summary()
    
    return model


In [59]:
!pip install scikit-image

In [60]:
## Task1a) Lung segmentation in chest X-ray images:
import os
from random import shuffle
from skimage.io import imread
from skimage.transform import resize


image_path = '/DL_course_data/Lab3/MRI/Image' 
mask_path = '/DL_course_data/Lab3/MRI/Mask'



In [61]:
def load_data(image_path,mask_path):
    
    image_list = os.listdir(image_path)
    mask_list = os.listdir(mask_path)
    
    images = []
    masks = []

    for image in image_list:
        img = imread(os.path.join(image_path, image), as_gray=True)  # "as_grey"
        img = resize(img, (240, 240), anti_aliasing=True).astype('float32')
        images.append(img)

    for mask in mask_list:
        mask_img = imread(os.path.join(mask_path, mask), as_gray=True)
        mask_img = resize(mask_img, (240, 240), anti_aliasing=True).astype('float32')
        masks.append(mask_img)
        
    ## Load data in traditional way
    # img_train, img_val, mask_train, mask_val = train_test_split(images, masks, shuffle = True,
    #                                                   test_size = 0.2)

    images = np.expand_dims(images, axis = -1)
    images = np.array(images)

    masks = np.expand_dims(masks, axis = -1)
    masks = np.array(masks)
    
    return images, masks
    

In [62]:
from tensorflow.keras import backend as K
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 0.0001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.0001)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [63]:
n_base =8
LR = 1e-4
batch_normalization = True
dropout = True
epochs = 5
Metric= 'Dice Coefficient'
batch_size = 8

img_w, img_h = 240,240
img_ch = 1




In [64]:
images, masks = load_data(image_path,mask_path)

In [13]:
from scipy.ndimage.morphology import binary_dilation, binary_erosion

mask_dilated = np.zeros(masks.shape)
mask_eroded = np.zeros(masks.shape)

for i in range(len(masks)):
    radius = 2
    structure = np.ones((round(radius*2+1),round(radius*2+1)))
    mask_dilated[i,:,:,0] = binary_dilation(masks[i,:,:,0], structure)
    mask_eroded[i,:,:,0] = binary_erosion(masks[i,:,:,0],structure)
weight_boundary = mask_dilated.astype(int) - mask_eroded.astype(int)



<ipython-input-13-e2c642e8f4a1>:1: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_dilation, binary_erosion
<ipython-input-13-e2c642e8f4a1>:1: DeprecationWarning: Please use `binary_erosion` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_dilation, binary_erosion


In [21]:
weight_boundary.shape

(9403, 240, 240, 1)

In [ ]:
def weighted_loss(weight_map, weight_strength): 
    def weighted_dice_loss(y_true, y_pred):
        y_true_f = K.flatten(y_true) 
        y_pred_f = K.flatten(y_pred)
        weight_f = K.flatten(weight_map) 
        weight_f = weight_f * weight_strength 
        weight_f = weight_f + 1
        weighted_intersection = K.sum(weight_f * (y_true_f * y_pred_f))
        return -(2. * weighted_intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return weighted_dice_loss

In [ ]:
def combine_generator(gen1, gen2, gen3): 
    while True:
        x = gen1.next() 
        y = gen2.next() 
        w = gen3.next() 
        yield([x, w], y)
def generator_with_weights(x_train, y_train, weights_train, batch_size):
    data_gen_args = dict(rotation_range=10.,
                            width_shift_range=0.1, 
                            height_shift_range=0.1, 
                            zoom_range=0.2, 
                            horizontal_flip=True)
    image_datagen = ImageDataGenerator(**data_gen_args) 
    mask_datagen = ImageDataGenerator(**data_gen_args) 
    weights_datagen = ImageDataGenerator(**data_gen_args)

    image_generator = image_datagen.flow(x_train, shuffle=False, batch_size=batch_size,seed=1)
    mask_generator = mask_datagen.flow(y_train, shuffle=False, batch_size=batch_size,seed=1)
    weight_generator = weights_datagen.flow(weights_train, shuffle=False, batch_size=batch_size,seed=1)
    
    train_generator = combine_generator(image_generator, mask_generator,weight_generator)
    return train_generator

In [ ]:


train_generator = generator_with_weights(x_train, y_train, weight_train, batch_size)

model_history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, 
            epochs=n_epoch, verbose=1, max_queue_size=1, 
            validation_steps=len(x_val), 
            validation_data=([x_val, weight_val], y_val), class_weight='auto')

In [47]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4)
cvscores = []


In [71]:
def generator(x_train, y_train, batch_size):
    n_train_sample = len(x_train)
    while True:
               
        for ind in (range(0, n_train_sample, batch_size)):
            
            batch_img = x_train[ind:ind+batch_size]
            batch_label = y_train[ind:ind+batch_size]
            
            # Sanity check assures batch size always satisfied
            # by repeating the last 2-3 images at last batch.
            length = len(batch_img)
            if length == batch_size:
                pass
            else:
                for tmp in range(batch_size - length):
                    batch_img = np.append(batch_img, np.expand_dims(batch_img[-1],axis=0), axis = 0)
                    batch_label = np.append(batch_label, np.expand_dims(batch_label[-1], axis=0), axis = 0)
        
            backgound_value = x_train.min()
            data_gen_args = dict(rotation_range=10.,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     cval = backgound_value,
                                     zoom_range=0.2,
                                     horizontal_flip = True)
            
            image_datagen = ImageDataGenerator(**data_gen_args)
            mask_datagen = ImageDataGenerator(**data_gen_args)
            
            image_generator = image_datagen.flow(batch_img, shuffle=False,
                                                 batch_size=batch_size,
                                                 seed=1)
            
            mask_generator = mask_datagen.flow(batch_label, shuffle=False,
                                               batch_size=batch_size,
                                               seed=1)
            
            image = image_generator.next()
            label = mask_generator.next()
            
            yield image, label

In [72]:
from matplotlib import image

for train, val in kf.split(images, masks):
    image_train, image_val = images[train],images[val]
    mask_train, mask_val = masks[train], masks[val]
    ## Get model
    model = get_unet(img_w, img_h, img_ch, n_base, LR, 
                batch_normalization, dropout)

    ## Add augmentation
    image_aug = ImageDataGenerator(rotation_range=10,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range = 0.2,
                                    horizontal_flip=True)

    train_generator = image_aug.flow(
                                    image_train,mask_train,
                                    batch_size = batch_size)

    val_generator = image_aug.flow(
                                    image_val,mask_val,
                                    batch_size = batch_size)


    # Compile the model
    model.compile(loss = [dice_coef_loss],          # Model Compiling   
                optimizer = Adam(lr = LR),
                metrics = [dice_coef, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    model.fit_generator(train_generator, steps_per_epoch = train_generator.n//batch_size,
                validation_data = val_generator, validation_steps = val_generator.n//batch_size,
                epochs = epochs,  verbose=1)


    # evaluate the model
    scores = model.evaluate(image_val, mask_val, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 240, 240, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_95 (Conv2D)             (None, 240, 240, 8)  80          ['input_6[0][0]']                
                                                                                                  
 batch_normalization_90 (BatchN  (None, 240, 240, 8)  32         ['conv2d_95[0][0]']              
 ormalization)                                                                                    
                                                                                            

<ipython-input-72-628a63948e9c>:31: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch = train_generator.n//batch_size,


881/881 [==============================] - 75s 51ms/step - loss: 0.9553 - dice_coef: 0.0447 - precision_5: 0.0343 - recall_5: 0.9115 - val_loss: 0.9531 - val_dice_coef: 0.0469 - val_precision_5: 0.0425 - val_recall_5: 0.9014
Epoch 2/5
881/881 [==============================] - 45s 51ms/step - loss: 0.9426 - dice_coef: 0.0574 - precision_5: 0.0540 - recall_5: 0.9301 - val_loss: 0.9658 - val_dice_coef: 0.0342 - val_precision_5: 0.0785 - val_recall_5: 4.3460e-05
Epoch 3/5
881/881 [==============================] - 45s 52ms/step - loss: 0.9328 - dice_coef: 0.0672 - precision_5: 0.0610 - recall_5: 0.8904 - val_loss: 0.9646 - val_dice_coef: 0.0354 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 4/5
881/881 [==============================] - 45s 51ms/step - loss: 0.9233 - dice_coef: 0.0767 - precision_5: 0.0639 - recall_5: 0.8555 - val_loss: 0.9613 - val_dice_coef: 0.0387 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 5/5
74/74 [==============================] -

In [73]:
cvscores

[2.9252661392092705, 10.042130947113037, 4.18890155851841, 3.611527383327484]

In [74]:
model.metrics_names

['loss', 'dice_coef', 'precision_8', 'recall_8']